In [1]:
import pandas as pd
import numpy as np

In [2]:
# 샘플 데이터프레임 생성
dict1 = {'item' : ['radio', 'computer','telephone'], 'price' : ['10000원', '30000원', '20000']}
df1 = pd.DataFrame(dict1)
df1

,item,price
0,radio,10000원
1,computer,30000원
2,telephone,20000


In [3]:
# 데이터 클렌징
df1.price = df1.price.str.replace('원','')  # 문자열 내 원을 공백으로 치환
df1.price = pd.to_numeric(df1.price)  # 숫자형으로 변환
print(df1.dtypes)
df1

item     object
price     int64
dtype: object


,item,price
0,radio,10000
1,computer,30000
2,telephone,20000


In [4]:
df1.price2 = df1.price.replace({10000 : '쌈', 20000 : '보통', 30000 : '비쌈'})  ## 처음 선언할 때는 메서드처럼 쓰면 안됨
df1

C:\Users\홍련\AppData\Local\Temp\ipykernel_46024\3561813521.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df1.price2 = df1.price.replace({10000 : '쌈', 20000 : '보통', 30000 : '비쌈'})  ## 처음 선언할 때는 메서드처럼 쓰면 안됨


,item,price
0,radio,10000
1,computer,30000
2,telephone,20000


In [5]:
df1['price2'] = df1.price.replace({10000 : '쌈', 20000 : '보통', 30000 : '비쌈'})  # 문자열 전체 매칭시
df1

,item,price,price2
0,radio,10000,쌈
1,computer,30000,비쌈
2,telephone,20000,보통


In [6]:
def condition(x):  # 함수 및 apply 활용 가능
    if x<20000:
        return '2만미만'
    else:
        return '2만이상'
df1.price.apply(condition)

0    2만미만
1    2만이상
2    2만이상
Name: price, dtype: object

In [7]:
df1['price'].apply(lambda x: x**2)

0    100000000
1    900000000
2    400000000
Name: price, dtype: int64

In [8]:
df1.price.transform([lambda x: np.log(x), np.sqrt])  # 복수열로도 사용 가능

,<lambda>,sqrt
0,9.210340,100.000000
1,10.308953,173.205081
2,9.903488,141.421356


In [9]:
df1.eval('price = price * 50')  # eval 메서드 활용

,item,price,price2
0,radio,500000,쌈
1,computer,1500000,비쌈
2,telephone,1000000,보통


In [10]:
# 샘플 데이터프레임 생성
dict2 = {'ipo' : ['witch factory', 'cubox','trueN'], 'deposit' : ['32,000,000,000', '22,500,000,000', '30,000,000,000']}
df2 = pd.DataFrame(dict2)
df2

,ipo,deposit
0,witch factory,"32,000,000,000"
1,cubox,"22,500,000,000"
2,trueN,"30,000,000,000"


In [11]:
# 천단위 소수점 삭제 후 숫자형 바꾸기
df2.deposit = df2.deposit.str.replace(',','').astype(float)
print(df2.dtypes)
df2

ipo         object
deposit    float64
dtype: object


,ipo,deposit
0,witch factory,3.200000e+10
1,cubox,2.250000e+10
2,trueN,3.000000e+10


In [12]:
# np.where 활용
np.where(df2.deposit >= 3e10, 1, 2)

array([1, 2, 1])

In [21]:
# 샘플 데이터프레임 생성
dict3 = {'Name' : ['John Harbor', 'San Miguel','Sir Otto'], 'Korean' : [60, 80, 70], 'Math' : [90,80,80], 'Society' : [70, 90, 95]}
df3 = pd.DataFrame(dict3)
df3

,Name,Korean,Math,Society
0,John Harbor,60,90,70
1,San Miguel,80,80,90
2,Sir Otto,70,80,95


In [22]:
# 원하는 위치에 컬럼 추가
df3.insert(3, 'English', [80, 90, 85])
df3

,Name,Korean,Math,English,Society
0,John Harbor,60,90,80,70
1,San Miguel,80,80,90,90
2,Sir Otto,70,80,85,95


In [23]:
# 하나의 컬럼을 둘로 쪼개기
df3[['First name', 'Last name']] = df3.Name.str.split(' ', expand=True)
df3

,Name,Korean,Math,English,Society,First name,Last name
0,John Harbor,60,90,80,70,John,Harbor
1,San Miguel,80,80,90,90,San,Miguel
2,Sir Otto,70,80,85,95,Sir,Otto


In [24]:
df3.columns

Index(['Name', 'Korean', 'Math', 'English', 'Society', 'First name',
       'Last name'],
      dtype='object')

In [25]:
# 컬럼 위치 자유자재로 바꾸기
col1 = df3.columns[-2:].to_list()
col2 = df3.columns[:-2].to_list()
col = col1+ col2
df3 = df3[col]
df3

,First name,Last name,Name,Korean,Math,English,Society
0,John,Harbor,John Harbor,60,90,80,70
1,San,Miguel,San Miguel,80,80,90,90
2,Sir,Otto,Sir Otto,70,80,85,95


In [31]:
# 컬럼 순번으로 변경하는 것도 가능
df3.columns = df3.columns[[2, 0, 1, 3, 4, 5, 6]]
df3

,Name,First name,Last name,Korean,Math,English,Society
0,John,Harbor,John Harbor,60,90,80,70
1,San,Miguel,San Miguel,80,80,90,90
2,Sir,Otto,Sir Otto,70,80,85,95


In [33]:
# 컬럼 순서를 거꾸로
df3[reversed(df3.columns)]

,Society,English,Math,Korean,Last name,First name,Name
0,70,80,90,60,John Harbor,Harbor,John
1,90,90,80,80,San Miguel,Miguel,San
2,95,85,80,70,Sir Otto,Otto,Sir


In [40]:
# List Comprehension : 서로 맞바꾸기
df3[['First name' if c == 'Last name' else 'Last name' if c=='First name' else c for c in df3.columns]]

,Name,Last name,First name,Korean,Math,English,Society
0,John,John Harbor,Harbor,60,90,80,70
1,San,San Miguel,Miguel,80,80,90,90
2,Sir,Sir Otto,Otto,70,80,85,95


In [56]:
tmp = df3.columns.to_list()
print(tmp)

['Name', 'First name', 'Last name', 'Korean', 'Math', 'English', 'Society']


In [58]:
print(tmp.remove('Society'))

ValueError: list.remove(x): x not in list

In [59]:
tmp

['Name', 'First name', 'Last name', 'Korean', 'Math', 'English']

In [51]:
df3.columns.to_list().remove('First name')

In [44]:
df3b

,Name,First name,Last name,Korean,Math,English,Society
0,John,Harbor,John Harbor,60,90,80,70
1,San,Miguel,San Miguel,80,80,90,90
2,Sir,Otto,Sir Otto,70,80,85,95


In [60]:
df3['Sum'] = df3.sum(axis=1, numeric_only=None)
df3

C:\Users\홍련\AppData\Local\Temp\ipykernel_46024\1440619615.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df3['Sum'] = df3.sum(axis=1, numeric_only=None)
C:\Users\홍련\AppData\Local\Temp\ipykernel_46024\1440619615.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Sum'] = df3.sum(axis=1, numeric_only=None)


,Name,First name,Last name,Korean,Math,English,Society,Sum
0,John,Harbor,John Harbor,60,90,80,70,300
1,San,Miguel,San Miguel,80,80,90,90,340
2,Sir,Otto,Sir Otto,70,80,85,95,330


In [61]:
df3.columns

Index(['Name', 'First name', 'Last name', 'Korean', 'Math', 'English',
       'Society', 'Sum'],
      dtype='object')

In [71]:
df4 = df3[df3.columns + 'Sum']
df4

KeyError: "None of [Index(['NameSum', 'First nameSum', 'Last nameSum', 'KoreanSum', 'MathSum',\n       'EnglishSum', 'SocietySum', 'SumSum'],\n      dtype='object')] are in the [columns]"

In [ ]:
df3.columns.tolist()

In [70]:
df3.columns.append('Sum']

TypeError: all inputs must be Index

In [67]:
df3

,Name,First name,Last name,Korean,Math,English,Society,Sum
0,John,Harbor,John Harbor,60,90,80,70,300
1,San,Miguel,San Miguel,80,80,90,90,340
2,Sir,Otto,Sir Otto,70,80,85,95,330
